In [2]:
!sudo apt update
!sudo apt install gcc



Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,200 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,374 kB]

In [1]:
!wget "https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/MatData.txt"


--2024-10-21 18:57:57--  https://raw.githubusercontent.com/arrondavide/HPC/refs/heads/main/Resources/MatData.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14116 (14K) [text/plain]
Saving to: ‘MatData.txt’

MatData.txt         100%[===================>]  13.79K  --.-KB/s    in 0.001s  

2024-10-21 18:57:57 (22.4 MB/s) - ‘MatData.txt’ saved [14116/14116]



In [3]:
%%writefile matrix_multiplication.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>


typedef struct {
    int rows;
    int cols;
    double **data;
} Matrix;

typedef struct {
    Matrix *A;
    Matrix *B;
    Matrix *C;
    int start_row;
    int end_row;
} ThreadArg;


Matrix* create_matrix(int rows, int cols);
void free_matrix(Matrix *mat);
int read_matrices(const char *filename, Matrix ***matrices, int *num_matrices);
void write_result(const char *filename, Matrix *C);
void* multiply_matrices_thread(void *arg);
void multiply_matrices(Matrix *A, Matrix *B, Matrix *C, int num_threads);

Matrix* create_matrix(int rows, int cols) {
    Matrix *mat = (Matrix*)malloc(sizeof(Matrix));
    mat->rows = rows;
    mat->cols = cols;
    mat->data = (double**)malloc(rows * sizeof(double*));
    for (int i = 0; i < rows; i++) {
        mat->data[i] = (double*)calloc(cols, sizeof(double));
    }
    return mat;
}

void free_matrix(Matrix *mat) {
    for (int i = 0; i < mat->rows; i++) {
        free(mat->data[i]);
    }
    free(mat->data);
    free(mat);
}

int read_matrices(const char *filename, Matrix ***matrices, int *num_matrices) {
    FILE *file = fopen(filename, "r");
    if (file == NULL) {
        fprintf(stderr, "Error: Cannot open file %s\n", filename);
        return 0;
    }

    *num_matrices = 0;
    *matrices = NULL;
    int capacity = 0;

    while (!feof(file)) {
        int rows, cols;
        if (fscanf(file, "%d,%d", &rows, &cols) != 2) {
            break;
        }

        if (*num_matrices >= capacity) {
            capacity = capacity == 0 ? 2 : capacity * 2;
            *matrices = realloc(*matrices, capacity * sizeof(Matrix*));
        }

        (*matrices)[*num_matrices] = create_matrix(rows, cols);
        Matrix *mat = (*matrices)[*num_matrices];

        for (int i = 0; i < rows; i++) {
            for (int j = 0; j < cols; j++) {
                if (fscanf(file, "%lf,", &(mat->data[i][j])) != 1) {
                    fprintf(stderr, "Error: Invalid data format in file\n");
                    fclose(file);
                    return 0;
                }
            }
        }

        (*num_matrices)++;
    }

    fclose(file);
    return 1;
}

void write_result(const char *filename, Matrix *C) {
    FILE *file = fopen(filename, "a");
    if (file == NULL) {
        fprintf(stderr, "Error: Cannot open file %s for writing\n", filename);
        exit(1);
    }

    fprintf(file, "%d,%d\n", C->rows, C->cols);
    for (int i = 0; i < C->rows; i++) {
        for (int j = 0; j < C->cols; j++) {
            fprintf(file, "%.6lf", C->data[i][j]);
            if (j < C->cols - 1) fprintf(file, ",");
        }
        fprintf(file, "\n");
    }
    fprintf(file, "\n");

    fclose(file);
}

void* multiply_matrices_thread(void *arg) {
    ThreadArg *thread_arg = (ThreadArg*)arg;
    Matrix *A = thread_arg->A;
    Matrix *B = thread_arg->B;
    Matrix *C = thread_arg->C;

    for (int i = thread_arg->start_row; i < thread_arg->end_row; i++) {
        for (int j = 0; j < B->cols; j++) {
            C->data[i][j] = 0.0;
            for (int k = 0; k < A->cols; k++) {
                C->data[i][j] += A->data[i][k] * B->data[k][j];
            }
        }
    }

    return NULL;
}

void multiply_matrices(Matrix *A, Matrix *B, Matrix *C, int num_threads) {
    pthread_t threads[num_threads];
    ThreadArg thread_args[num_threads];
    int rows_per_thread = A->rows / num_threads;
    int remaining_rows = A->rows % num_threads;

    int start_row = 0;
    for (int i = 0; i < num_threads; i++) {
        thread_args[i].A = A;
        thread_args[i].B = B;
        thread_args[i].C = C;
        thread_args[i].start_row = start_row;
        thread_args[i].end_row = start_row + rows_per_thread + (i < remaining_rows ? 1 : 0);

        pthread_create(&threads[i], NULL, multiply_matrices_thread, &thread_args[i]);

        start_row = thread_args[i].end_row;
    }

    for (int i = 0; i < num_threads; i++) {
        pthread_join(threads[i], NULL);
    }
}

int main(int argc, char *argv[]) {
    if (argc != 3) {
        fprintf(stderr, "Usage: %s <input_file> <num_threads>\n", argv[0]);
        return 1;
    }

    const char *input_file = argv[1];
    int num_threads = atoi(argv[2]);

    Matrix **matrices;
    int num_matrices;

    if (!read_matrices(input_file, &matrices, &num_matrices)) {
        return 1;
    }

    if (num_matrices % 2 != 0) {
        fprintf(stderr, "Error: Odd number of matrices in input file\n");
        return 1;
    }

    // Clear the output file
    fclose(fopen("output.txt", "w"));

    for (int i = 0; i < num_matrices; i += 2) {
        Matrix *A = matrices[i];
        Matrix *B = matrices[i+1];

        if (A->cols != B->rows) {
            fprintf(stderr, "Error: Matrix dimensions do not match for multiplication (%dx%d * %dx%d)\n",
                    A->rows, A->cols, B->rows, B->cols);
            continue;
        }

        Matrix *C = create_matrix(A->rows, B->cols);

        int max_threads = A->rows > B->cols ? A->rows : B->cols;
        int actual_threads = num_threads > max_threads ? max_threads : num_threads;

        multiply_matrices(A, B, C, actual_threads);

        write_result("output.txt", C);

        free_matrix(C);
    }

    // Free all matrices
    for (int i = 0; i < num_matrices; i++) {
        free_matrix(matrices[i]);
    }
    free(matrices);

    return 0;
}

Writing matrix_multiplication.c


In [4]:
# Compile the C program
!gcc -o matrix_multiplication matrix_multiplication.c -pthread


In [5]:
# Run the C program with the input file 'MatData.txt' and specify the number of threads (e.g., 4)
!./matrix_multiplication MatData.txt  4



In [6]:
# Display the contents of the result file
!cat output.txt


2,2
9947047195.529209,14200596809.529045
10519836149.444487,16034405692.067020

9,9
31011369938.519295,35692486797.110977,40653841394.870621,37119512263.544342,42842953929.597488,39459046936.607040,30211185997.392277,46541455110.334343,34304520492.786846
24080248746.934837,30950047968.443741,23444667214.924980,29827064060.174530,20632508745.592812,35857950656.252678,18558586495.492565,31712132456.134750,24437891253.850525
29950962965.743343,32449599240.077686,29768991455.971081,34601572745.962837,27019244485.234554,30997729980.889591,26253744109.200367,33139957790.751572,28919154317.116528
38604584517.468719,31099655203.781036,38702026565.196518,33331277042.610878,30242896894.484764,31965815857.285477,30244593998.380314,41378186062.584160,31494416169.893806
44799877528.651306,38475111809.887062,42816954148.718597,36034108397.048973,36939615796.087875,42481989740.043259,28141513831.482491,45637538469.009041,35726962203.193069
42915661548.738022,34918647181.886810,42301124885.394119,3205